# Cohorts analysis

## Problem

We are launching a recommendation tool that identifies vendor cohorts and suggests performance improvements based on peer comparisons within those cohorts. Our current cohort definition uses a six-level nested hierarchy (Country → City → Area → Price → Cuisine → Grade) that was designed primarily for explainability to account managers rather than analytical rigor.

This ad hoc approach creates two risks for our MVP rollout:

Weak statistical foundation: We haven't validated whether our cohorts actually group similar-performing vendors together Stakeholder confidence: Without a principled justification for cohort boundaries, leadership are questioning recommendation validity

Our SVP has specified that cohorts must be "sensible and comparable" - meaning they should be both statistically meaningful and intuitive to business stakeholders. We need a methodology that validates our current hierarchy against this standard and provides a framework for refinement.

## Solution
To identify which vendor characteristics create the most "comparable" cohorts, we'll use a three-step approach: 

1. **dimension ranking through regression analysis** to find the dimensions that best predict vendor performance 
2. **hierarchy optimization**, to find the hierarchy that best groups vendors into similar performing vendors. 
The measure will help identify better cohort separation - vendors within cohorts are similar while cohorts differ meaningfully 
3. **cluster validation and refinement** to ensure that the final hierarchy is sensible and comparable.

## Data 

Collected in `create_data.ipynb`

In [1]:
from datetime import date
from highlight_text import fig_text
from numpy.linalg import LinAlgError

import os
import numpy as np
import pandas as pd
import pyfixest as pf
import matplotlib.pyplot as plt
import bigframes.pandas as bpd
import statsmodels.api as sm
import seaborn as sns

import warnings
warnings.filterwarnings(action='once')

%load_ext google.cloud.bigquery
bpd.options.bigquery.project = "dhh-ncr-stg"

/Users/patrick.doupe/DH/ad_hoc/cohorts/.venv/lib/python3.13/site-packages/google/cloud/bigquery/__init__.py:237: FutureWarning: %load_ext google.cloud.bigquery is deprecated. Install bigquery-magics package and use `%load_ext bigquery_magics`, instead.
  warnings.warn(


In [2]:
%%bigquery df
SELECT * FROM `dhh-ncr-stg.patrick_doupe.cohort_vendor_base`

Query is running:   0%|          |

Downloading:   0%|          |

In [3]:
%%bigquery df_current
SELECT * FROM `dhh-ncr-stg.patrick_doupe.current_cohort_vendor_base`

Query is running:   0%|          |

Downloading:   0%|          |

In [4]:
df.groupby('created_month')['total_orders_gmv'].agg(lambda x: x.isna().mean())

created_month
2024-07-01    1.000000
2024-08-01    1.000000
2024-09-01    1.000000
2024-10-01    1.000000
2024-11-01    1.000000
2024-12-01    0.566383
2025-01-01    0.395856
2025-02-01    0.374108
2025-03-01    0.365466
2025-04-01    0.340335
2025-05-01    0.316548
2025-06-01    0.304840
2025-07-01    0.466223
Name: total_orders_gmv, dtype: float64

Let's drop everything prior to December 2024. They're all NaNs anyway

In [5]:
df_all = df.loc[df.created_month >= pd.to_datetime("2024-12-01")]
df = df_all.loc[df_all.created_month <= pd.to_datetime("2025-01-01")].copy()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 695800 entries, 2 to 4514484
Data columns (total 20 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   region                    695800 non-null  object 
 1   entity_id                 695800 non-null  object 
 2   vendor_code               695800 non-null  object 
 3   created_month             695800 non-null  dbdate 
 4   chain_id                  385823 non-null  object 
 5   chain_name                385821 non-null  object 
 6   entity                    695800 non-null  object 
 7   city                      695800 non-null  object 
 8   area                      695800 non-null  object 
 9   vendor_grade              695800 non-null  object 
 10  is_new_vendor             695800 non-null  boolean
 11  cuisine                   695800 non-null  object 
 12  key_account_sub_category  695800 non-null  object 
 13  successful_orders         360986 non-null  Int64

In [7]:
# bloody decimal
df['total_orders_gmv'] = df['total_orders_gmv'].astype("float")
df['successful_orders_gmv'] = df['successful_orders_gmv'].astype("float")

## Summary statistics

In [8]:
df.describe()

,successful_orders,total_orders,new_customer_orders,retained_customers,successful_customers,total_orders_gmv,successful_orders_gmv
count,360986.0,360986.0,360986.0,360986.0,360986.0,360986.000000,360986.000000
mean,182.054592,229.706986,64.547149,77.848991,135.635327,3286.702115,2680.541892
std,597.440846,665.759736,186.281656,269.165901,411.529284,10121.649159,9249.670584
min,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000
25%,2.0,6.0,1.0,0.0,2.0,84.100000,21.620000
50%,15.0,32.0,7.0,6.0,14.0,434.765000,212.135000
75%,109.0,169.0,49.0,41.0,90.0,2270.990000,1528.530000
max,34990.0,35441.0,18070.0,15740.0,20789.0,630683.280000,623058.730000


In [9]:
df[df.describe().columns].mean() / df[df.describe().columns].median()

successful_orders        12.136973
total_orders              7.178343
new_customer_orders       9.221021
retained_customers       12.974832
successful_customers      9.688238
total_orders_gmv          7.559721
successful_orders_gmv    12.636019
dtype: Float64

- We can see that the distributions are heavily right skewed.
- The mean is 7-13x the median
- Outliers are default!

In [10]:
df.vendor_grade.value_counts()

vendor_grade
NA     316886
D      206370
C       93207
B       41877
A       26603
AA       8146
AAA      2711
Name: count, dtype: int64

A lot of unknown vendor grades

In [11]:
df.cuisine.value_counts()

cuisine
UNK                     101585
Desserts                 29055
Arabic                   25591
Pizza                    23734
Kebap & Türk Mutfağı     23134
                         ...  
Food & Drink                 2
Pho                          2
Bagely                       2
Tatarák                      2
Specialty Store              1
Name: count, Length: 418, dtype: int64

A lot of unkonwn cuisines

In [12]:
df['average_price'] = df['successful_orders_gmv'] / df['successful_orders'] 

In [13]:
df.head()

,region,entity_id,vendor_code,created_month,chain_id,chain_name,entity,city,area,vendor_grade,...,cuisine,key_account_sub_category,successful_orders,total_orders,new_customer_orders,retained_customers,successful_customers,total_orders_gmv,successful_orders_gmv,average_price
2,MENA,HS_SA,21052,2025-01-01,1,McDonald's KSA,hs_sa,Riyadh,Uraija Al Gharbiyah,AA,...,Fast Food,Global KA,2148,2197,506,1145,1623,39992.01,39076.42,18.192002
13,MENA,HS_SA,21135,2025-01-01,9964,Al Qalaa Al Raqiah,hs_sa,Jeddah,Al Amir Fawaz Al Janouby,C,...,Saudi,UNK,331,333,132,146,268,6018.09,5982.96,18.075408
16,MENA,HS_SA,21170,2025-01-01,9976,Signature,hs_sa,Hail,al shabili el qarbi,D,...,Juices,National KA,223,225,104,93,187,3128.45,3105.24,13.924843
17,MENA,HS_SA,21172,2025-01-01,9976,Signature,hs_sa,Riyadh,Al Maather,C,...,Juices,National KA,466,472,187,178,341,7485.45,7385.10,15.847854
23,MENA,HS_SA,21187,2025-01-01,9976,Signature,hs_sa,Dammam,Ar Rakah Ash Shamaliyah,B,...,Juices,National KA,776,779,232,361,568,12945.84,12889.24,16.609845


In [19]:
df['noise'] = np.random.randn(df.shape[0])

In [20]:
cols_to_check = [
    'region',
    'entity',
    'city',
    'area',
    'vendor_grade',
    'key_account_sub_category',
    'cuisine'
]

continuous_cols = [
    'average_price',
    'retained_customers',
    'new_customer_orders'
]

In [21]:
df.fillna({'successful_orders_gmv': 0}, inplace=True)

for col in continuous_cols:
    df.fillna({col: 0}, inplace=True)

In [22]:
df[cols_to_check] = df[cols_to_check].astype("category")
df[continuous_cols] = df[continuous_cols].astype("float")

for col in continuous_cols:
    df.loc[:, f'{col}_QQ'] = pd.qcut(df[col], q=4, duplicates='drop')

unique_cats = {}
for col in [col_ + '_QQ' for col_ in continuous_cols]:
    tmp = col + '_id'
    df[tmp], cats = pd.factorize(df[col])
    df[tmp].astype('category')
    cols_to_check.append(tmp)
    unique_cats[tmp] = cats


/var/folders/c_/xt8d51196jl46y09mq3j_3_w0000gq/T/ipykernel_58753/51835288.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[(12.537, 648.52], (12.537, 648.52], (12.537, 648.52], (12.537, 648.52], (12.537, 648.52], ..., (12.537, 648.52], (12.537, 648.52], (12.537, 648.52], (12.537, 648.52], (12.537, 648.52]]
Length: 695800
Categories (2, interval[float64, right]): [(-0.001, 12.537] < (12.537, 648.52]]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:, f'{col}_QQ'] = pd.qcut(df[col], q=4, duplicates='drop')


In [23]:
def get_regression_output(df, y_column, group_column, poisson=False):
    if poisson:
        fit = pf.fepois(f"{y_column} ~ noise | {group_column}", data = df)
    else:
        fit = pf.feols(f"{y_column} ~ noise | {group_column}", data = df)
    
    return {'adj_r2' : fit._adj_r2, 'residuals' : fit.resid()}

In [24]:
results = {col: get_regression_output(df, 'successful_orders_gmv', col) for col in cols_to_check}

/Users/patrick.doupe/DH/ad_hoc/cohorts/.venv/lib/python3.13/site-packages/pyfixest/estimation/feols_.py:2604: RuntimeWarning: divide by zero encountered in matmul
  tXX = X.T @ X
/Users/patrick.doupe/DH/ad_hoc/cohorts/.venv/lib/python3.13/site-packages/pyfixest/estimation/feols_.py:2604: RuntimeWarning: overflow encountered in matmul
  tXX = X.T @ X
/Users/patrick.doupe/DH/ad_hoc/cohorts/.venv/lib/python3.13/site-packages/pyfixest/estimation/feols_.py:2604: RuntimeWarning: invalid value encountered in matmul
  tXX = X.T @ X
/Users/patrick.doupe/DH/ad_hoc/cohorts/.venv/lib/python3.13/site-packages/pyfixest/estimation/feols_.py:501: RuntimeWarning: divide by zero encountered in matmul
  self._tZX = _Z.T @ _X
/Users/patrick.doupe/DH/ad_hoc/cohorts/.venv/lib/python3.13/site-packages/pyfixest/estimation/feols_.py:501: RuntimeWarning: overflow encountered in matmul
  self._tZX = _Z.T @ _X
/Users/patrick.doupe/DH/ad_hoc/cohorts/.venv/lib/python3.13/site-packages/pyfixest/estimation/feols_.py:

In [32]:
results_ar2 = { k: v['adj_r2'] for k, v in results.items()}
results_ar2 = pd.DataFrame(list(results_ar2.items()), columns=['Cohort Dimension', 'Adjusted R2']).sort_values(by='Adjusted R2', ascending=False)
results_ar2['Adjusted R2'] = results_ar2['Adjusted R2'].round(3)
results_ar2

,Cohort Dimension,Adjusted R2
4,vendor_grade,0.226
8,retained_customers_QQ_id,0.125
9,new_customer_orders_QQ_id,0.123
7,average_price_QQ_id,0.062
3,area,0.041
5,key_account_sub_category,0.038
6,cuisine,0.036
2,city,0.035
1,entity,0.032
0,region,0.001


In [33]:
results_ar2

,Cohort Dimension,Adjusted R2
4,vendor_grade,0.226
8,retained_customers_QQ_id,0.125
9,new_customer_orders_QQ_id,0.123
7,average_price_QQ_id,0.062
3,area,0.041
5,key_account_sub_category,0.038
6,cuisine,0.036
2,city,0.035
1,entity,0.032
0,region,0.001


In [27]:
results_ar2.to_json('output/ranking_results.json')

In [ ]:
df_resid = pd.DataFrame(residuals)

In [54]:
df[cols_to_check] = df[cols_to_check].astype("category")

In [57]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from itertools import combinations

# Get all categorical columns
cat_cols = ['region', 'entity', 'city', 'area', 'vendor_grade', 
           'key_account_sub_category', 'cuisine', 'average_price_QQ_id', 'retained_customers_QQ_id', 'new_customer_orders_QQ_id']

def analyze_pairwise_relationships(df, cat_cols):
    """Analyze all pairwise relationships"""
    results = []
    
    for col1, col2 in combinations(cat_cols, 2):
        crosstab = pd.crosstab(df[col1], df[col2])
        
        # Summary stats
        n_combinations = (crosstab > 0).sum().sum()  # Non-zero cells
        max_possible = len(df[col1].cat.categories) * len(df[col2].cat.categories)
        sparsity = 1 - (n_combinations / max_possible)
        
        results.append({
            'var1': col1,
            'var2': col2, 
            'actual_combinations': n_combinations,
            'max_possible': max_possible,
            'sparsity': sparsity,
            'cramers_v': cramers_v(crosstab)  # Association strength
        })
    
    return pd.DataFrame(results).sort_values('cramers_v', ascending=False)

def cramers_v(crosstab):
    """Calculate Cramer's V for association strength"""
    chi2 = chi2_contingency(crosstab)[0]
    n = crosstab.sum().sum()
    return np.sqrt(chi2 / (n * (min(crosstab.shape) - 1)))



In [62]:
analyze_pairwise_relationships(df, ['average_price_QQ_id', 'retained_customers_QQ_id', 'new_customer_orders_QQ_id', 'vendor_grade'])

,var1,var2,actual_combinations,max_possible,sparsity,cramers_v
3,retained_customers_QQ_id,new_customer_orders_QQ_id,4,4,0.0,0.836416
4,retained_customers_QQ_id,vendor_grade,14,14,0.0,0.576839
5,new_customer_orders_QQ_id,vendor_grade,14,14,0.0,0.507983
0,average_price_QQ_id,retained_customers_QQ_id,4,4,0.0,0.471730
1,average_price_QQ_id,new_customer_orders_QQ_id,4,4,0.0,0.466464
2,average_price_QQ_id,vendor_grade,14,14,0.0,0.464341


In [58]:
# Step 1: Understand pairwise relationships
pairwise_results = analyze_pairwise_relationships(df, cat_cols)
print("Strongest associations:")
print(pairwise_results.head())
print("Weakest associations:")
print(pairwise_results.tail())

Strongest associations:
      var1     var2  actual_combinations  max_possible  sparsity  cramers_v
0   region   entity                   15            60  0.750000   1.000000
1   region     city                  911          3644  0.750000   1.000000
9   entity     city                  943         13665  0.930992   0.998917
5   region  cuisine                  476          1672  0.715311   0.888376
10  entity     area                 5521         80565  0.931471   0.839691
Weakest associations:
      var1                       var2  actual_combinations  max_possible  \
20    city                    cuisine                12145        380798   
11  entity               vendor_grade                  105           105   
3   region               vendor_grade                   28            28   
8   region  new_customer_orders_QQ_id                    8             8   
7   region   retained_customers_QQ_id                    8             8   

    sparsity  cramers_v  
20  0.968106   

In [59]:
def find_complementary_dimensions(df, cat_cols, threshold=0.3):
    """Find pairs with LOW Cramer's V (independent information)"""
    results = []
    
    for col1, col2 in combinations(cat_cols, 2):
        crosstab = pd.crosstab(df[col1], df[col2])
        v = cramers_v(crosstab)
        
        results.append({
            'var1': col1,
            'var2': col2,
            'cramers_v': v,
            'independence_score': 1 - v,  # Higher = more independent
            'interpretation': interpret_relationship(v)
        })
    
    # Sort by independence (lowest V first)
    independent_pairs = pd.DataFrame(results).sort_values('cramers_v')
    
    print("=== MOST INDEPENDENT PAIRS (Additional Information) ===")
    return independent_pairs[independent_pairs['cramers_v'] < threshold]

def interpret_relationship(v):
    if v < 0.1: return "Independent (great for cohorts)"
    elif v < 0.3: return "Weakly related (good for cohorts)"  
    elif v < 0.6: return "Moderately related (some overlap)"
    else: return "Highly related (redundant)"

In [60]:
def find_optimal_dimension_set(df, cat_cols, max_dims=5):
    """Find set of dimensions with minimal overlap"""
    
    # Calculate all pairwise V values
    v_matrix = {}
    for col1, col2 in combinations(cat_cols, 2):
        crosstab = pd.crosstab(df[col1], df[col2])
        v_matrix[(col1, col2)] = cramers_v(crosstab)
    
    # Greedy algorithm: start with most independent pairs
    selected_dims = []
    remaining_dims = cat_cols.copy()
    
    while len(selected_dims) < max_dims and remaining_dims:
        if not selected_dims:
            # Start with dimension with most independent relationships
            independence_scores = {}
            for dim in remaining_dims:
                avg_v = np.mean([v_matrix.get((dim, other), v_matrix.get((other, dim), 0)) 
                               for other in remaining_dims if other != dim])
                independence_scores[dim] = 1 - avg_v
            
            best_start = max(independence_scores.items(), key=lambda x: x[1])[0]
            selected_dims.append(best_start)
            remaining_dims.remove(best_start)
        else:
            # Add dimension with lowest average V to selected dimensions
            best_addition = None
            lowest_avg_v = float('inf')
            
            for candidate in remaining_dims:
                avg_v_with_selected = np.mean([
                    v_matrix.get((candidate, selected), v_matrix.get((selected, candidate), 0))
                    for selected in selected_dims
                ])
                
                if avg_v_with_selected < lowest_avg_v:
                    lowest_avg_v = avg_v_with_selected
                    best_addition = candidate
            
            if best_addition and lowest_avg_v < 0.5:  # Threshold for "acceptable" independence
                selected_dims.append(best_addition)
                remaining_dims.remove(best_addition)
            else:
                break
    
    return selected_dims, lowest_avg_v

In [61]:
# Find your best orthogonal dimensions
cat_cols = ['region', 'entity', 'city', 'area', 'vendor_grade', 
           'key_account_sub_category', 'cuisine']

# Get independence analysis
independent_pairs = find_complementary_dimensions(df, cat_cols)
print(independent_pairs.head(10))

# Find optimal dimension set
optimal_dims, avg_overlap = find_optimal_dimension_set(df, cat_cols, max_dims=4)
print(f"\nOptimal dimensions: {optimal_dims}")
print(f"Average overlap (Cramer's V): {avg_overlap:.3f}")

# Validate your choice
print("\n=== VALIDATION: Overlap within chosen set ===")
for i, dim1 in enumerate(optimal_dims):
    for dim2 in optimal_dims[i+1:]:
        crosstab = pd.crosstab(df[dim1], df[dim2])
        v = cramers_v(crosstab)
        print(f"{dim1} ↔ {dim2}: V = {v:.3f}")

=== MOST INDEPENDENT PAIRS (Additional Information) ===
            var1                      var2  cramers_v  independence_score  \
3         region              vendor_grade   0.088502            0.911498   
8         entity              vendor_grade   0.126755            0.873245   
14          city                   cuisine   0.147027            0.852973   
4         region  key_account_sub_category   0.172860            0.827140   
17          area                   cuisine   0.179427            0.820573   
15          area              vendor_grade   0.180056            0.819944   
12          city              vendor_grade   0.198966            0.801034   
18  vendor_grade  key_account_sub_category   0.220440            0.779560   
19  vendor_grade                   cuisine   0.224298            0.775702   
9         entity  key_account_sub_category   0.233631            0.766369   

                       interpretation  
3     Independent (great for cohorts)  
8   Weakly relat